In [1]:
import os
import numpy as np
from dataclasses import dataclass, fields
from VASP_job.code.main import VASP_job

import pandas as pd
import ase
from ase.spacegroup import crystal

from pyiron.atomistics.structure.atoms import ase_to_pyiron
from pyiron.atomistics.structure.atoms import pyiron_to_ase

import matplotlib.pyplot as plt


# Define a custom formatting function for vectors in pandas
format_vector = lambda vector: '[' + ', '.join(['{:.10f}'.format(x) for x in vector]) + ']'

In [2]:
a = 3.95
alpha = 90
structure = crystal(('Ni', 'Mn', 'N'),
                       basis=[(0.00, 0.00, 0.00), (0.00, 0.50, 0.50), (0.50, 0.50, 0.50)],
                       spacegroup=221,
                       cellpar=[a, a, a, alpha, alpha, alpha])
structure.symbols.species()

{'Mn', 'N', 'Ni'}

In [3]:
Dtheta = 120 * np.pi/180
theta = -Dtheta

number_of_atoms = len(structure)
magdirs   = []
ms        = []
B_CONSTR  = []
for i in range(number_of_atoms):
   if 'Mn' == structure.get_chemical_symbols()[i]:
      theta += Dtheta
      magdirs.append([np.cos(theta), np.sin(theta), 0])
      ms.append(0.5)
      B_CONSTR.append([0, 0, 0])
   else:
      magdirs.append([0, 0, 0])
      ms.append(1)
      B_CONSTR.append([0, 0, 0])

In [4]:
structure.new_array("magdirs", magdirs, dtype=float)
structure.new_array("ms", ms, dtype=float)
structure.new_array("B_CONSTR", B_CONSTR, dtype=float)

In [5]:
structure = structure.repeat((2,1,1))
# structure = ase_to_pyiron(structure)
# structure.set_repeat([2,2,2])
# structure = pyiron_to_ase(structure)

In [6]:
structure.arrays["ms"]

array([1. , 0.5, 0.5, 0.5, 1. , 1. , 0.5, 0.5, 0.5, 1. ])

In [7]:
structure.arrays["positions"]

array([[0.   , 0.   , 0.   ],
       [0.   , 1.975, 1.975],
       [1.975, 0.   , 1.975],
       [1.975, 1.975, 0.   ],
       [1.975, 1.975, 1.975],
       [3.95 , 0.   , 0.   ],
       [3.95 , 1.975, 1.975],
       [5.925, 0.   , 1.975],
       [5.925, 1.975, 0.   ],
       [5.925, 1.975, 1.975]])

In [8]:
structure.get_chemical_symbols()

['Ni', 'Mn', 'Mn', 'Mn', 'N', 'Ni', 'Mn', 'Mn', 'Mn', 'N']

In [9]:
# Create a defition that does all this?

executable_path= "/home/emendive/workbench/work/from_claix/codes/VASP/vasp.5.4.4-flag4/bin"
potential_path = "/home/emendive/workbench/work/from_claix/codes/VASP/potentials/potpaw_PBE"
vasp = VASP_job(verbose="low", executable_path=executable_path, potential_path=potential_path)

In [10]:
vasp.set_calculation(structure)

In [11]:
structure.get_chemical_symbols()

['Ni', 'Mn', 'Mn', 'Mn', 'N', 'Ni', 'Mn', 'Mn', 'Mn', 'N']

In [12]:
elements = list( vasp.df["elements"] )

In [14]:
species = []
for element in elements:
   if element not in species:
      species.append(element)

species

['Mn', 'N', 'Ni']

In [25]:
tuple(vasp.df["positions"].values)[-1]

array([3.95, 0.  , 0.  ])

In [16]:
from ase.visualize import view
view(structure)

<Popen: returncode: None args: ['/home/emendive/anaconda3/envs/work_pyiron/b...>

In [17]:
import nglview
view = nglview.show_ase(structure)
view.add_unitcell()
view.add_ball_and_stick()
view

NGLWidget()

In [ ]:
def vasp_next(vasp, dir_0, name):

   cwd = dir_0 + "/" + name
   
   cwd_next = cwd + "_k2x2x2"
   os.chdir(dir_0)
   run("cp -r "+cwd+"_k1x1x1"+" "+cwd_next)
   os.chdir(cwd_next)

   # update inputs and run
   vasp.io.cwd = cwd_next

   vasp.structure.kpoints = "2 2 2"
   vasp.io.write_KPOINTS(kpoints)

   vasp.io.INCAR.ISTART = 1
   vasp.io.INCAR.ICHARG = 1
   species = vasp.structure.species
   magmoms   = vasp.df["magmoms"].tolist()
   B_CONSTRs = vasp.df["B_CONSTRs"].tolist()
   vasp.io.write_INCAR(species, magmoms, B_CONSTRs)

   vasp.run_vasp()

   return vasp

def prepare_vasp(dir_0, A, mu, repeat_cell, name, m, next_flag=False):
   # prepare folder
   cwd_first = cwd + "_k1x1x1"
   if not os.path.exists(cwd):
      run("mkdir " + cwd_first, shell=True)
   os.chdir(cwd_first)

   # prepare crystal structure
   a = 3.95 # A
   alpha = 90
   structure = crystal((A, 'Mn', 'N'),
                        basis=[(0.00, 0.00, 0.00), (0.00, 0.50, 0.50), (0.50, 0.50, 0.50)],
                        spacegroup=221,
                        cellpar=[a, a, a, alpha, alpha, alpha])

   # prepare magnetism (triangular AFM state)
   Dtheta = 120 * np.pi/180
   theta = -Dtheta

   number_of_atoms = len(structure)
   magdirs   = []
   ms    = []
   # B_CONSTRs = []
   for i in range(number_of_atoms):
      if 'Mn' == structure.get_chemical_symbols()[i]:
         theta += Dtheta
         magdirs.append(np.array([np.cos(theta), np.sin(theta), 0])*mu)
         ms.append(value_of_m)
         # B_CONSTRs.append([0.0, 0.0, 0.0])
      else:
         magdirs.append([0.0, 0.0, 0.0])
         ms.append(1)
         # B_CONSTRs.append([0.0, 0.0, 0.0])

   structure.new_array("magdirs", magdirs, dtype=float)
   structure.new_array("ms", ms, dtype=float)
   # structure.new_array("B_CONSTR", B_CONSTR, dtype=float)

   # create vasp class
   vasp = VASP_job(verbose="low", executable_path=executable_path, potential_path=potential_path, pyscript=True)
   vasp.io.cwd = cwd_first

   # create SUPERCELL (should be done after adding arrays):
   structure = structure.repeat(repeat_cell)

   # update inputs
   vasp.structure.kpoints = "1 1 1"
   vasp.io.INCAR.NPAR = "10"
   vasp.io.INCAR.NELM = "100"

   # set calculation and run
   vasp.set_calculation(structure)
   vasp.run_vasp()

   return vasp